# Welcome to the Lab 🥼🧪

## Housing Mix Analysis

Why does housing mix matter? The types of units trading varies by market and can provide insights into the market's health. This analysis will look at the housing mix of a market and compare it to the national average. The [Parcl Labs Price Feeds](https://www.parcllabs.com/articles/parcl-labs-price-feed-whitepaper) is the price per square foot of units trading on a market. Understanding the mix and the variation in mix over time can provide insight into where prices could go. 

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/traders/housing_mix_analysis.ipynb)

In [ ]:
# Environment setup
import os
import sys
import subprocess
from datetime import datetime

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido numpy
    !git clone https://github.com/ParclLabs/parcllabs-examples.git
    sys.path.append('/content/parcllabs-examples/python/')
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')
    cur_dir = os.getcwd()
    chart_dir = os.path.join(cur_dir, '..')
    sys.path.append(chart_dir)

In [ ]:
import parcllabs
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from charting.utils import create_labs_logo_dict, format_metro_names

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
# init client
client = ParclLabsClient(api_key=api_key)

In [ ]:
# Get top 100 CBSAs by population
markets = client.search_markets.retrieve(
    as_dataframe=True,
    sort_by='PARCL_EXCHANGE_MARKET',
    sort_order='DESC',
    params={
        'limit': 14
    }
)

miami = client.search_markets.retrieve(
    parcl_id=5352987,
    as_dataframe=True
)

markets = pd.concat([markets, miami])
markets

In [ ]:
def clean_names(nme):
    replace = {
        'Washington City': 'D.C.',
        'United States Of America': 'USA',
        'New York City': 'NYC',
        'Kings County': 'Brooklyn, NY',
    }
    if nme in replace.keys():
        return replace[nme]
    else:
        return nme
    
markets['name'] = markets['name'].apply(clean_names)
markets

In [ ]:
feeds = client.price_feed.retrieve_many(
        parcl_ids=markets['parcl_id'].tolist(),
        start_date='2019-01-01',
        as_dataframe=True,
        params={'limit': 1000},  # expand the limit to 1000, these are daily series
        auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)

feeds.head()

In [ ]:
# get the current housing stock

stock = client.market_metrics_housing_stock.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    as_dataframe=True,
    params={
        'limit': 300 # get current state
    }
)

In [ ]:
stock['pct_single_family'] = stock['single_family'] / stock['all_properties'] 
stock['pct_condo'] = stock['condo'] / stock['all_properties'] 
stock['pct_townhouse'] = stock['townhouse'] / stock['all_properties'] 
stock['pct_other'] = stock['other'] / stock['all_properties'] 

In [ ]:
# join name
df = stock.merge(markets[['name', 'parcl_id']], on='parcl_id')
df['name'] = df['name'].apply(lambda x: format_metro_names(x, include_state=True))


In [ ]:
import plotly.graph_objects as go
import pandas as pd


def build_pie_chart(
        data,
        market_name: str,
        save_path: str = None
):
    # Prepare data for the pie chart
    labels = ['Single Family', 'Condo', 'Townhouse', 'Other']
    values = [data['single_family'].values[0], data['condo'].values[0], data['townhouse'].values[0], data['other'].values[0]]

    # Create pie chart
    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=values,
        textinfo='label+percent',
        textposition='outside',
        pull=[0.05, 0.05, 0.05, 0.05],
        marker=dict(
            colors=['#636EFA', '#EF553B', '#00CC96', '#AB63FA'],
            line=dict(color='#FFFFFF', width=2)
        ),
        insidetextorientation='radial'
    )])

    # load assets
    labs_logo_dict = create_labs_logo_dict(
        src='labs',
        y=1.015,
        sizex=0.3,
        sizey=0.3
    )

    # Add Parcl Labs logo
    fig.add_layout_image(
        labs_logo_dict
    )

    # Add title and layout adjustments
    fig.update_layout(
        title_text=f'Housing Stock Mix: {market_name}',
        title_font_size=24,
        title_x=0.5,
        annotations=[dict(
            text=f'Total Units: {data["all_properties"].values[0]:,}',
            x=0.5,
            y=-0.1,
            showarrow=False,
            font=dict(size=14, color='white')
        )],
        showlegend=False,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )

    # Add hover info
    fig.update_traces(
        hoverinfo='label+percent+value',
        textfont_size=14
    )

    # Adjust figure size
    fig.update_layout(
        autosize=False,
        width=800,
        height=800
    )

    # Save the figure
    if save_path:
        fig.write_image(save_path)

    # Show the figure
    fig.show()


In [ ]:
prop_types = [
    'SINGLE_FAMILY',
    'CONDO',
    'TOWNHOUSE',
    'ALL_PROPERTIES'
]

alldf = []

for ptype in prop_types:

    mix = client.market_metrics_housing_event_counts.retrieve_many(
        parcl_ids=markets['parcl_id'].tolist(),
        as_dataframe=True,
        property_type=ptype,
        params={
            'limit': 300
        }
    )

    # mix = mix.rename(columns={
    #     'sales': f'{ptype.lower()}_sales',
    #     'new_listings_for_sale': f'{ptype.lower()}_new_listings_for_sale',
    #     'new_rental_listings': f'{ptype.lower()}_new_rental_listings',
    # })
    mix['property_type'] = ptype.lower()
    alldf.append(mix[['parcl_id', 'date', 'sales', 'property_type']])
    # alldf.append(mix)

In [ ]:
chart = pd.concat(alldf[0:3])
chart = chart.merge(alldf[-1][['parcl_id', 'date', 'sales']].rename(columns={'sales': 'all_sales'}), on=['parcl_id', 'date'])
chart

In [ ]:
sfh = chart.loc[chart['property_type']=='single_family']
all_other = sfh.copy(deep=True)
all_other['sales'] = all_other['all_sales']-all_other['sales']
all_other['property_type'] = 'All Other Units'
chart = pd.concat([sfh, all_other])
chart['property_type'] = chart['property_type'].replace({'single_family': 'Single Family'})

In [ ]:
chart['pct_sales'] = chart['sales'] / chart['all_sales']
chart = chart.merge(markets[['name', 'parcl_id']], on='parcl_id')


In [ ]:
# load assets
labs_logo_dict = create_labs_logo_dict(
    src='labs',
    y=1.015,
    sizex=0.15,
    sizey=0.15,
    x=0.47
)

In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Define the function to build the line chart
def build_line_chart(
    data: pd.DataFrame,
    title: str = None,
    x_axis_title: str = None,
    y_axis_title: str = None,
    save_path: str = None,
    value_name: str = None
):
    HEIGHT = 900
    WIDTH = 1600
    
    fig = go.Figure()

    # Define colors for each property type
    colors = {
        'Single Family': '#57A3FF',  # Blue
        'All Other Units': '#FF7F0E',  # Orange
        'Townhouse': '#2CA02C'       # Green
    }

    # Plot each property type
    for property_type in data['property_type'].unique():
        subset = data[data['property_type'] == property_type]
        fig.add_trace(go.Scatter(
            x=subset['date'],
            y=subset[value_name],
            mode='lines',
            name=property_type,
            line=dict(width=2, color=colors.get(property_type, '#FFFFFF'))  # Default to white if no color found
        ))

    # Add vertical dotted line
    fig.add_shape(
        dict(
            type="line",
            x0="2020-03-01",
            y0=0,
            x1="2020-03-01",
            y1=1,
            xref='x',
            yref='paper',
            line=dict(
                color="Red",
                width=2,
                dash="dot",
            )
        )
    )

    # Add annotation for the vertical line
    fig.add_annotation(
        dict(
            x="2020-03-01",
            y=1,
            xref='x',
            yref='paper',
            xanchor='left',  # Position the text to the right of the line
            xshift=10,  # Shift the text slightly to the right
            text="COVID Starts",
            showarrow=False,
            font=dict(
                size=12,
                color="Red"
            ),
            align="center"
        )
    )

    # Add logo image
    fig.add_layout_image(
        labs_logo_dict
    )
    
    fig.update_layout(
        margin=dict(l=0, r=0, t=110, b=0),
        height=HEIGHT,
        width=WIDTH,
        title={
            'text': title,
            'y': 0.99,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=28, color='#FFFFFF'),
        },
        plot_bgcolor='#000000',  # Dark background for better contrast
        paper_bgcolor='#000000',  # Dark background for the paper
        font=dict(color='#FFFFFF'),
        xaxis=dict(
            title_text=x_axis_title,
            showgrid=False,  # Disable vertical grid lines
            tickangle=-45,
            tickfont=dict(size=14),
            linecolor='rgba(255, 255, 255, 0.7)',  # Axis line color with opacity
            linewidth=1  # Axis line width
        ),
        yaxis=dict(
            title_text=y_axis_title,
            showgrid=True,
            gridwidth=0.5,  # Horizontal grid line width
            gridcolor='rgba(255, 255, 255, 0.2)',  # Horizontal grid line color with opacity
            tickfont=dict(size=14),
            tickprefix='',  # Remove dollar sign prefix
            ticksuffix='%',  # Add percentage suffix
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',  # Axis line color with opacity
            linewidth=1  # Axis line width
        ),
        hovermode='x unified',  # Unified hover mode for better interactivity
        hoverlabel=dict(
            bgcolor='#1F1F1F',
            font_size=14,
            font_family="Rockwell"
        ),
        legend=dict(
            x=0,
            y=0,
            traceorder='normal',
            font=dict(
                size=12,
                color='#FFFFFF'
            ),
            bgcolor='rgba(0,0,0,0)',
            bordercolor='rgba(0,0,0,0)'
        )
    )

    if save_path:
        fig.write_image(save_path, width=WIDTH, height=HEIGHT)
    
    # Show the plot
    fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Define the function to build the combined line chart
def build_combined_line_chart(
    data_main: pd.DataFrame,
    data_secondary: pd.DataFrame,
    title_main: str = None,
    x_axis_title: str = None,
    y_axis_title_main: str = None,
    y_axis_title_secondary: str = None,
    save_path: str = None,
    value_name_main: str = None,
    value_name_secondary: str = None
):
    HEIGHT_MAIN = 720
    HEIGHT_SECONDARY = 180
    TOTAL_HEIGHT = HEIGHT_MAIN + HEIGHT_SECONDARY
    WIDTH = 1600

    # Create subplots: 2 rows, 1 column
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        row_heights=[0.8, 0.2],
        vertical_spacing=0.02
    )

    # Define colors for each property type
    colors = {
        'Single Family': '#57A3FF',  # Blue
        'All Other Units': '#FF7F0E',  # Orange
        'Townhouse': '#2CA02C'       # Green
    }

    # Plot each property type for the main data
    for property_type in data_main['property_type'].unique():
        subset = data_main[data_main['property_type'] == property_type]
        fig.add_trace(
            go.Scatter(
                x=subset['date'],
                y=subset[value_name_main],
                mode='lines',
                name=property_type,
                line=dict(width=2, color=colors.get(property_type, '#FFFFFF'))  # Default to white if no color found
            ),
            row=1, col=1
        )

    # Add vertical dotted line for main chart
    fig.add_shape(
        dict(
            type="line",
            x0="2020-03-01",
            y0=0,
            x1="2020-03-01",
            y1=1,
            xref='x',
            yref='y1',
            line=dict(
                color="Red",
                width=2,
                dash="dot",
            )
        ),
        row=1, col=1
    )

    # Add annotation for the vertical line
    fig.add_annotation(
        dict(
            x="2020-03-01",
            y=0.95,
            xref='x',
            yref='y1',
            xanchor='left',  # Position the text to the right of the line
            xshift=10,  # Shift the text slightly to the right
            text="COVID Starts",
            showarrow=False,
            font=dict(
                size=12,
                color="Red"
            ),
            align="center"
        ),
        row=1, col=1
    )

    # Plot the secondary data
    fig.add_trace(
        go.Scatter(
            x=data_secondary['date'],
            y=data_secondary[value_name_secondary],
            mode='lines',
            name='Secondary Data',
            line=dict(width=2, color='#FFD700')  # Gold for the secondary series
        ),
        row=2, col=1
    )

    fig.update_layout(
        height=TOTAL_HEIGHT,
        width=WIDTH,
        title={
            'text': title_main,
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=28, color='#FFFFFF'),
        },
        plot_bgcolor='#000000',  # Dark background for better contrast
        paper_bgcolor='#000000',  # Dark background for the paper
        font=dict(color='#FFFFFF'),
        xaxis=dict(
            title_text=x_axis_title,
            showgrid=False,  # Disable vertical grid lines
            tickangle=-45,
            tickfont=dict(size=14),
            linecolor='rgba(255, 255, 255, 0.7)',  # Axis line color with opacity
            linewidth=1  # Axis line width
        ),
        yaxis=dict(
            title_text=y_axis_title_main,
            showgrid=True,
            gridwidth=0.5,  # Horizontal grid line width
            gridcolor='rgba(255, 255, 255, 0.2)',  # Horizontal grid line color with opacity
            tickfont=dict(size=14),
            tickprefix='',  # Remove dollar sign prefix
            ticksuffix='%',  # Add percentage suffix
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',  # Axis line color with opacity
            linewidth=1  # Axis line width
        ),
        yaxis2=dict(
            title_text=y_axis_title_secondary,
            showgrid=True,
            gridwidth=0.5,  # Horizontal grid line width
            gridcolor='rgba(255, 255, 255, 0.2)',  # Horizontal grid line color with opacity
            tickfont=dict(size=14),
            tickprefix='$',  # Dollar sign prefix
            ticksuffix='',  # No suffix
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',  # Axis line color with opacity
            linewidth=1  # Axis line width
        ),
        hovermode='x unified',  # Unified hover mode for better interactivity
        hoverlabel=dict(
            bgcolor='#1F1F1F',
            font_size=14,
            font_family="Rockwell"
        ),
        legend=dict(
            x=0,
            y=-0.2,
            traceorder='normal',
            font=dict(
                size=12,
                color='#FFFFFF'
            ),
            bgcolor='rgba(0,0,0,0)',
            bordercolor='rgba(0,0,0,0)'
        )
    )

    if save_path:
        fig.write_image(save_path, width=WIDTH, height=TOTAL_HEIGHT)
    
    # Show the plot
    fig.show()

# Example usage
pid = 5826765
line_chart_data = chart.loc[chart['parcl_id'] == pid]
feed_data = feeds.loc[feeds['parcl_id'] == pid]
build_combined_line_chart(
    data_main=line_chart_data,
    data_secondary=feed_data,
    x_axis_title='',
    y_axis_title_main='Percentage of Sales',
    y_axis_title_secondary='Price per Square Foot',
    value_name_main='pct_sales',
    value_name_secondary='price_feed',
)


In [ ]:
chart['parcl_id'].unique()

In [ ]:
# Example usage
# Assuming you have a DataFrame named 'chart' with columns 'parcl_id', 'property_type', 'date', 'pct_sales', and 'name'
for pid in chart['parcl_id'].unique():
    pie_chart = df.loc[(df['parcl_id']==pid) & (df['date']=='2024-04-01')]
    line_chart_data = chart.loc[chart['parcl_id'] == pid]
    name = line_chart_data['name'].values[0]
    build_pie_chart(
        data=pie_chart,
        market_name=name,
        save_path=f'../graphics/housing_mix_analysis/{name}_housing_stock_mix.png'
    )
    build_line_chart(
        data=line_chart_data,
        title=f'Mix of Home Sales by Property Type: {name}',
        x_axis_title='',
        y_axis_title='Percentage of Home Sales',
        value_name='pct_sales',
        save_path=f'../graphics/housing_mix_analysis/{name}_home_sales_mix.png'
    )